In [ ]:
import pandas as pd
import json

In [ ]:
#Reading Excel file
customers = pd.read_excel("/content/drive/MyDrive/PIE_assignment/Customer.xls")
#Reading csv file
orders = pd.read_csv("/content/drive/MyDrive/PIE_assignment/Order.csv")
#Reading .json file
with open("/content/drive/MyDrive/PIE_assignment/Shipping.json") as f:
    shipping = pd.json_normalize(json.load(f))

In [ ]:
def data_profile(df):
    stats = []
    total_count = len(df)

    for col_name, data_type in df.dtypes.items():

        populated_count = df[col_name].count()
        # Completeness check / null records
        null_count = df[col_name].isnull().sum()
        # Uniqueness check / distinct records
        distinct_count = df[col_name].nunique()
        # Records with leading/trailing white spaces
        white_spaces = df[col_name].astype(str).str.strip() != df[col_name].astype(str)
        white_spaces_count = white_spaces.sum()
        # Records with special characters
        special_character = df[col_name].astype(str).str.contains(r"[!@#$%^&*(),?\":{}|<>]", regex=True, na=False)
        special_character_count = special_character.sum()
        # Count of values with string "NaN", "NAN", "nan", "Nan"
        nan_like_values = df[col_name].astype(str).str.lower().isin(["nan"]).sum()

        # Min and Max length for all data types
        lengths = df[col_name].astype(str).str.len()
        min_length = lengths.min()
        max_length = lengths.max()

        stats.append([
            col_name, data_type, total_count, populated_count, null_count,
            distinct_count, white_spaces_count,
            special_character_count, nan_like_values,
            min_length, max_length
        ])

    columns = [
        "Column_name", "Data_Type", "Total_Records", "Populated_count", "Null_count",
        "Distinct_count", "Records_with_white_spaces", "Records_with_special_chars",
        "String_NaN_count", "Min_length", "Max_length"
    ]

    return pd.DataFrame(stats, columns=columns)

Checking data quality of customers record


In [ ]:
data_profile(customers)

,Column_name,Data_Type,Total_Records,Populated_count,Null_count,Distinct_count,Records_with_white_spaces,Records_with_special_chars,String_NaN_count,Min_length,Max_length
0,Customer_ID,int64,250,250,0,250,0,0,0,1,3
1,First,object,250,250,0,171,0,5,0,3,11
2,Last,object,250,250,0,189,0,0,0,2,10
3,Age,int64,250,250,0,62,0,0,0,2,2
4,Country,object,250,250,0,3,0,0,0,2,3


In [ ]:
#Checking additional 5 records with special character
customers[customers['First'].str.contains(r'[!@#$%^&*(),?\":{}|<>]', regex=True, na=False)].head(10)


,Customer_ID,First,Last,Age,Country
5,6,N!cole,Jones,33,USA
13,14,N!cole,Lara,77,UK
161,162,N!cole,Bennett,51,USA
170,171,L@rry,Cole,50,USA
213,214,N!cole,Mcintyre,18,UK


In [ ]:
#Checking for first name with length 11
customers[customers['First'].str.len() == 11].head(10)

,Customer_ID,First,Last,Age,Country
146,147,Christopher,Mcdonald,43,USA
194,195,Christopher,Doyle,43,UAE
225,226,Christopher,Robinson,23,USA
244,245,Christopher,Miles,30,UAE


In [ ]:
#Checking for last name with length 10
customers[customers['Last'].str.len() == 10].head(10)

,Customer_ID,First,Last,Age,Country
7,8,Jason,Montgomery,58,UK
110,111,Audrey,Richardson,53,USA
127,128,Sabrina,Mclaughlin,50,UK
217,218,Thomas,Mclaughlin,29,UK


**Quality analysis of Customers data**

1.   All 250 records are populated, ensuring data completeness.


1.   There are no missing records, confirming the dataset is complete.
2.   The "First" attribute contains 5 records with special characters, which may require cleaning to ensure data accuracy.












**Checking data quality of orders data**

In [ ]:
data_profile(orders)

,Column_name,Data_Type,Total_Records,Populated_count,Null_count,Distinct_count,Records_with_white_spaces,Records_with_special_chars,String_NaN_count,Min_length,Max_length
0,Order_ID,int64,250,250,0,250,0,0,0,1,3
1,Item,object,250,250,0,8,0,0,0,5,8
2,Amount,int64,250,250,0,9,0,0,0,3,5
3,Customer_ID,int64,250,250,0,160,0,0,0,1,3


In [ ]:
#Checking for distinct records
orders["Item"].drop_duplicates().head(10)

,Item
0,Keyboard
1,Mouse
2,Monitor
4,Mousepad
5,Harddisk
6,Webcam
8,DDR RAM
9,Headset


In [ ]:
#Checking for the clustering of amount
orders["Amount"].drop_duplicates().head(10)

,Amount
0,400
1,300
2,12000
4,250
5,5000
6,350
7,200
8,1500
9,900


**Quality analysis of Orders data**

1. All 250 records are populated, ensuring data completeness.

2. There are no missing records, confirming the dataset is complete.

3. The Item attribute has 8 distinct values, which indicates a limited product variety.

4. The Amount attribute has 9 distinct values, suggesting that amounts are grouped or standardized.

5. Orders data have 61 Customers record without any shipping ID.


**Checking data quality of shipping data**

In [ ]:
data_profile(shipping)

,Column_name,Data_Type,Total_Records,Populated_count,Null_count,Distinct_count,Records_with_white_spaces,Records_with_special_chars,String_NaN_count,Min_length,Max_length
0,Shipping_ID,int64,250,250,0,250,0,0,0,1,3
1,Status,object,250,250,0,2,0,0,0,7,9
2,Customer_ID,int64,250,250,0,154,0,0,0,1,3


In [ ]:
#Checking for clustering of shipping id over customer_id
shipping.groupby('Customer_ID')['Shipping_ID'].count().sort_values(ascending=False).head(10)

,Shipping_ID
Customer_ID,
173,4
35,4
185,4
189,4
22,3
69,3
27,3
30,3
9,3


In [ ]:
#Checking for customer IDs present in Order data but not available in Shipping records
Order_shipping_join = pd.merge(orders, shipping, on='Customer_ID', how='left')
miss_customers = Order_shipping_join[Order_shipping_join['Shipping_ID'].isnull()]

In [17]:
Order_shipping_join2 = pd.merge(orders, shipping, on='Customer_ID', how='inner')

In [23]:
miss_customers['Customer_ID'].nunique()

61

In [24]:
#Checking for Customer IDs present in Shipping but not available in Order data
Shipping_order_join = pd.merge(shipping, orders, on='Customer_ID', how='left')
miss_customers2 = Shipping_order_join[Shipping_order_join['Order_ID'].isnull()]

In [25]:
miss_customers2['Customer_ID'].nunique()

55

In [27]:
miss_customers2.head(10)

,Shipping_ID,Status,Customer_ID,Order_ID,Item,Amount
0,1,Pending,173,NaN,NaN,NaN
1,2,Pending,155,NaN,NaN,NaN
6,5,Delivered,72,NaN,NaN,NaN
13,9,Pending,199,NaN,NaN,NaN
19,13,Delivered,141,NaN,NaN,NaN
20,14,Pending,155,NaN,NaN,NaN
29,20,Delivered,6,NaN,NaN,NaN
37,25,Delivered,2,NaN,NaN,NaN
39,27,Pending,58,NaN,NaN,NaN
40,28,Delivered,93,NaN,NaN,NaN


**Quality analysis of Shipping data**

1. All 250 records are populated, ensuring data completeness.

2. There are no missing records, confirming the dataset is complete.

3. The Status attribute has 2 distinct values, likely representing delivery statuses such as “Pending” and “Delivered”(can add additional flag for in progress shipping)

4. The Customer_ID attribute has 154 distinct values, indicating that multiple shipments are associated with some customers, while others have none.

5. **55 customers have shipping records but no corresponding orders**, which may indicate incomplete order data or special cases requiring further validation.

# **Creating Business reporting Script**



1.   **Total amount spent and the country for the Pending delivery status for each country.**




In [42]:
shipping[shipping['Status'] == 'Pending'] \
.merge(customers, on='Customer_ID', how='left') \
.merge(orders, on='Customer_ID', how='left') \
.groupby('Country')['Amount'] \
.sum() \
.reset_index() \
.rename(columns={'Amount': 'pending_amount'}) \
.sort_values(by='pending_amount', ascending=False) \
.head(10)

,Country,pending_amount
1,UK,136300.0
2,USA,65500.0
0,UAE,53800.0


2. Total quantity sold, and total amount spent for each customer, along with the product details.


In [51]:
orders.groupby('Customer_ID') \
.agg(
total_transactions = ('Order_ID', 'count'), #Assuming one order have one transaction
total_quantity = ('Item', 'count'),
total_amount = ('Amount', 'sum'),
product_details = ('Item', lambda x: list(x.unique()))) \
.reset_index() \
.sort_values(by='total_amount', ascending=False) \
.head()

,Customer_ID,total_transactions,total_quantity,total_amount,product_details
103,166,3,3,17350,"[Monitor, Harddisk, Webcam]"
80,129,2,2,17000,"[Monitor, Harddisk]"
76,123,2,2,17000,"[Harddisk, Monitor]"
55,96,4,4,14700,"[Mouse, Headset, DDR RAM, Monitor]"
120,193,4,4,13950,"[Monitor, Mousepad, DDR RAM]"


3.  Maximum product purchased for each country.

In [67]:
orders.merge(customers, on="Customer_ID", how="left") \
.groupby(["Country", "Item"]) \
.agg(total_purchased=("Order_ID", "count")) \
.reset_index() \
.sort_values(["Country", "total_purchased"], ascending=[True, False]) \
.groupby("Country") \
.head(1) \
.head()

,Country,Item,total_purchased
3,UAE,Keyboard,12
14,UK,Mousepad,24
22,USA,Mousepad,18


4. Most purchased product based on the age category less than 30 and above 30

In [74]:
orders.merge(customers, on="Customer_ID", how="left") \
.assign(age_category = lambda df: df["Age"].apply(lambda x: "Under 30" if x < 30 else "30 and above")) \
.groupby(["age_category", "Item"]) \
.agg(total_amount=("Amount", "sum")) \
.reset_index() \
.sort_values("total_amount", ascending=False) \
.groupby("age_category") \
.head(1) \
.head()

,age_category,Item,total_amount
4,30 and above,Monitor,204000
12,Under 30,Monitor,96000


5. country that had minimum transactions and sales amount.

In [79]:
orders.merge(customers, on="Customer_ID", how="left") \
.groupby("Country") \
.agg(
total_transactions=("Order_ID", "count"),
total_amount=("Amount", "sum")) \
.reset_index() \
.head(1)

,Country,total_transactions,total_amount
0,UAE,40,49950
